### Pricing delayed start options via simulation w/o and w/ conditional MC

This notebook is sample code on using conditional Monte Carlo to improve results compared to traditional Monte Carlo simulation.

In [2]:
#Imports 

import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from time import time
# import our BMS price and delta functions
from BMS import BMS_price, BMS_delta
# import ecdf function
from statsmodels.distributions.empirical_distribution import ECDF

In [3]:
# Define parameters

s0 = 100
t1 = 0.5
T = 1
sig = 0.15
r = 0.04
q = 0.015

lda = 0.9

In [4]:
# exact solution
c = s0 * np.exp(-q*t1) * BMS_price('call', 1, lda, r, q, sig, T, t1)
print('Exact solution: %3.4f' % c)

Exact solution: 11.6130


In [5]:
np.random.seed(123)
n_sims = 10000
discountedPayoffs1 = np.zeros(n_sims)
discountedPayoffs2 = np.zeros(n_sims)

In [6]:
st = time()

# could be made faster by vectorizing and removing the for loop
for i in range(n_sims):
    z1 = np.random.randn()
    s1 = s0 * np.exp((r - q -sig*sig/2)*t1 + sig*np.sqrt(t1)*z1)
    # set strike
    K = lda * s1
    # naive MC
    z2 = np.random.randn()
    s2 = s1 * np.exp((r - q -sig*sig/2)*(T - t1) + sig*np.sqrt(T - t1)*z2)
    discountedPayoffs1[i] = np.exp(-r*T) * max(s2 - K, 0)
    # conditional MC
    discountedPayoffs2[i] = np.exp(-r*t1) * BMS_price('call', s1, K, r, q, sig, T, t1)
    
c_hat1 = np.mean(discountedPayoffs1)
var_hat1 = np.var(discountedPayoffs1, ddof=1)

c_hat2 = np.mean(discountedPayoffs2)
var_hat2 = np.var(discountedPayoffs2, ddof=1)

et = time()
print('Elapsed time was %3.4f seconds.' % (et - st))

Elapsed time was 1.4375 seconds.


Pay attention to the half-width of CI

In [7]:
print('Exact solution: %3.4f' % c)
print('n_sims = %i' % n_sims)
print('Naive MC: %3.4f (%3.4f)' % (c_hat1, var_hat1))
print('Conditional MC: %3.4f (%3.4f)' % (c_hat2, var_hat2))

Exact solution: 11.6130
n_sims = 10000
Naive MC: 11.6677 (93.7672)
Conditional MC: 11.6362 (1.5492)


What we see is that the conditional MC estimator is closer to the value given by the black-scholes model, and more importantly, it has significantly less variance.